In [67]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import RidgeCV
import re

import pymc as pm
import arviz as az
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings("ignore")

In [69]:
def relative_ratings(lineup_df, players_df, year):
    # load in year and get average rating
    dfYear = pd.read_csv(f'/Users/austincoffelt/Documents/WNBA_Teams/wnba_teams_{year}.csv')
    avgRating = np.mean(dfYear['Offensive Rating'].astype(float))

    for i in lineup_df.index.to_list():
        lineup_df.loc[i, 'relOffRtg'] = lineup_df.loc[i, 'Offensive Rating'].astype(float) - avgRating
        lineup_df.loc[i, 'relDefRtg'] = avgRating - lineup_df.loc[i, 'Defensive Rating'].astype(float)

    for i in players_df.index.to_list():
        players_df.loc[i, 'relOffRtg'] = players_df.loc[i, 'Offensive Rating'].astype(float) - avgRating
        players_df.loc[i, 'relDefRtg'] = avgRating - players_df.loc[i, 'Defensive Rating'].astype(float)

In [71]:
PREVIOUS_ANSWERS = ['None']

In [73]:
def create_fullname_dummies(lineup_df, players_df, team, year):
    # Create first initial + last name to full names mapping (with duplicates)
    def get_first_initial_last(name):
        parts = name.split()
        if len(parts) >= 2:
            return f"{parts[0][0].upper()}. {parts[-1].upper()}"
        return name.upper()
    
    # Create mapping from "I. LASTNAME" format to possible full names
    initial_last_map = {}
    for name in players_df['Name']:
        key = get_first_initial_last(name)
        if key not in initial_last_map:
            initial_last_map[key] = []
        initial_last_map[key].append(name)
    
    # Position labels
    positions = {1: 'PG', 2: 'SG', 3: 'SF', 4: 'PF', 5: 'C'}
    
    # Process each lineup
    for idx, lineup in lineup_df.iterrows():
        lineup_display_num = idx + 1
        # Get names in format like "I. Lastname"
        initial_last_names = [name.strip() for name in lineup['Lineup'].split(',')]
        full_names = []
        
        for i, initial_last in enumerate(initial_last_names, 1):
            pos_num = i
            # Convert to uppercase to match players_df format
            lookup_key = initial_last.upper()
            possible_players = initial_last_map.get(lookup_key, [])
            
            # Unique case - auto select
            if len(possible_players) == 1:
                full_names.append(possible_players[0])
            
            # Duplicate case - check previous answers
            elif len(possible_players) > 1:
                answer_key = (team, year, lineup_display_num, pos_num, initial_last)
                if answer_key in PREVIOUS_ANSWERS:
                    choice = PREVIOUS_ANSWERS[answer_key]
                    full_names.append(possible_players[choice-1])
                    print(f"Auto-selected {possible_players[choice-1]} for {initial_last} in {team} {year} lineup {idx} position {pos_num}")
                    continue
                    
                # If no previous answer, prompt user
                print(f"\nConflict in lineup {idx+1} with {team}, {year}:")
                print(f"Position {pos_num} ({positions[pos_num]}): {initial_last}")
                print("Possible players:")
                for j, player in enumerate(possible_players, 1):
                    print(f"{j}. {player}")
                
                while True:
                    try:
                        choice = int(input("Enter the correct player number: "))
                        if 1 <= choice <= len(possible_players):
                            full_names.append(possible_players[choice-1])
                            break
                        else:
                            print("Invalid choice. Try again.")
                    except ValueError:
                        print("Please enter a number.")
            
            # Unknown player
            else:
                full_names.append(f"UNKNOWN {initial_last}")
        
        # Update the lineup with full names
        lineup_df.at[idx, 'FullNames'] = ', '.join(full_names)
    
    # Create dummy variables
    all_players = players_df['Name'].unique()
    for player in all_players:
        lineup_df[player] = lineup_df['FullNames'].str.contains(player).astype(int)
    
    return lineup_df

In [75]:
def enhanced_bayesian_orapm(years):
    RAPM = []
    problematic_cases = []
    
    for year in years:
        try:
            lineup_DF = pd.read_csv(f"/Users/austincoffelt/Documents/WNBA_lineups/wnba_lineups_{year}_complete.csv")
            players_DF = pd.read_csv(f"/Users/austincoffelt/Documents/WNBA_players/wnba_players_{year}.csv", dtype={'Total Minutes': float, 'relOffRtg': float})
            players_DF = players_DF.drop_duplicates()

            relative_ratings(lineup_DF, players_DF, year)

            teams = list(lineup_DF['Team'].unique())
        
            for team in teams:
                try:
                    lineup_df = lineup_DF[lineup_DF['Team'] == team]
                    lineup_df = lineup_df[lineup_df['Minutes'] >= 2]
                    players_df = players_DF[players_DF['Team'] == team].copy()

                    # Validate required columns exist
                    required_lineup_cols = {'Team', 'Minutes'}
                    required_player_cols = {'Team', 'Name', 'Total Minutes', 'relOffRtg'}
                    if not required_lineup_cols.issubset(lineup_DF.columns):
                        raise ValueError(f"Missing required columns in lineup data: {required_lineup_cols - set(lineup_DF.columns)}")
                    if not required_player_cols.issubset(players_DF.columns):
                        raise ValueError(f"Missing required columns in player data: {required_player_cols - set(players_DF.columns)}")

                    # For problematic years (like 2004), you might need special handling:
                    if year == 2004:
                        lineup_DF['Minutes'] = pd.to_numeric(lineup_DF['Minutes'], errors='coerce').fillna(0)
                        players_DF['Total Minutes'] = pd.to_numeric(players_DF['Total Minutes'], errors='coerce').fillna(1)
                        players_DF['relOffRtg'] = pd.to_numeric(players_DF['relOffRtg'], errors='coerce').fillna(0)
                        
                    # Ensure we have relOffRtg for all players
                    assert 'relOffRtg' in players_df.columns, "Missing relOffRtg in players_df"
                    
                    result_df = create_fullname_dummies(lineup_df.copy(), players_df, team, year)
                    player_columns = list(players_df['Name'])
                    result_df = result_df[player_columns + ['relOffRtg']].dropna(subset=['relOffRtg'])
                    
                    if len(result_df) < 20:
                        problematic_cases.append(f"{team} {year} (only {len(result_df)} lineups)")
                        continue
                    
                    X = result_df.drop('relOffRtg', axis=1).values
                    y = result_df['relOffRtg'].values
                    player_names = result_df.drop('relOffRtg', axis=1).columns.tolist()
                    
                    try:
                        print(f"Running informed model for {team} {year}")
                        trace = informed_model(X, y, player_names, players_df)
                    except Exception as e:
                        print(f"Informed model failed, trying robust version: {str(e)}")
                        trace = robust_informed_model(X, y, player_names, players_df)
                    
                    coef_dict = process_results(trace, team, year, player_names, players_df)
                    RAPM.append(coef_dict)
                    plot_player_effects(trace, player_names, team, year)
                    
                except Exception as e:
                    problematic_cases.append(f"{team} {year} ({str(e)})")
                    continue
    
        except Exception as e:
            problematic_cases.append(f"Year {year} failed: {str(e)}")
            continue
            
    if problematic_cases:
        print("\nSkipped/Problematic Cases:")
        for case in problematic_cases:
            print(f"- {case}")
    
    return RAPM

def informed_model(X, y, player_names, players_df):
    """Model using pre-calculated relOffRtg as priors"""
    with pm.Model() as model:
        # Get player priors and minutes
        player_priors = players_df.set_index('Name')['relOffRtg'].reindex(player_names).values
        player_minutes = players_df.set_index('Name')['Total Minutes'].reindex(player_names).fillna(1).values
        
        # Hierarchical variance
        sigma_league = pm.HalfNormal('sigma_league', sigma=5)
        
        # Player effects centered around their relOffRtg
        z = pm.Normal('z', mu=0, sigma=1, shape=len(player_names))
        prior_strength = np.log1p(player_minutes) / 2
        
        beta = pm.Deterministic('beta', player_priors + z * sigma_league / prior_strength)
        mu = pm.math.dot(X, beta)
        
        # Likelihood
        sigma_likelihood = pm.HalfNormal('sigma_likelihood', sigma=5)
        pm.Normal('obs', mu=mu, sigma=sigma_likelihood, observed=y)
        
        trace = pm.sample(
            3000, tune=3000,
            chains=4,
            target_accept=0.95,
            init='adapt_diag'
        )
    return trace

def robust_informed_model(X, y, player_names, players_df):
    """Robust version with Student-T likelihood and parameter constraints"""
    with pm.Model() as model:
        player_priors = players_df.set_index('Name')['relOffRtg'].reindex(player_names).values
        player_minutes = players_df.set_index('Name')['Total Minutes'].reindex(player_names).fillna(1).values
        
        sigma_league = pm.HalfNormal('sigma_league', sigma=5)
        
        z = pm.Normal('z', mu=0, sigma=1, shape=len(player_names))
        prior_strength = np.log1p(player_minutes) / 2
        
        # Constrained player effects
        beta = pm.Deterministic('beta',
                              pm.math.clip(
                                  player_priors + z * sigma_league / prior_strength,
                                  -25, 25  # Reasonable bounds for basketball RAPM
                              ))
        mu = pm.math.dot(X, beta)
        
        # Robust likelihood
        sigma_likelihood = pm.HalfNormal('sigma_likelihood', sigma=5)
        pm.StudentT('obs', nu=4, mu=mu, sigma=sigma_likelihood, observed=y)
        
        trace = pm.sample(
            2000, tune=2000,
            chains=4,
            target_accept=0.95,
            init='adapt_diag'
        )
    return trace

def process_results(trace, team, year, player_names, players_df):
    """Your existing results processing logic"""
    coef_dict = {
        'Team': team,
        'Year': year,
        'players': {},
        'diagnostics': az.summary(trace, var_names=['sigma_league', 'sigma_likelihood']).to_dict()
    }
    
    beta_samples = trace.posterior['beta'].values.reshape(-1, len(player_names))
    player_stats = players_df.set_index('Name')[['Total Minutes']].reindex(player_names)
    
    for i, player in enumerate(player_names):
        samples = beta_samples[:, i]
        coef_dict['players'][player] = {
            'mean': float(np.mean(samples)),
            'lower': float(np.percentile(samples, 2.5)),
            'upper': float(np.percentile(samples, 97.5)),
            'minutes': int(player_stats.loc[player, 'Total Minutes']),
            'reliability': 'high' if player_stats.loc[player, 'Total Minutes'] > 500 else 
                          'medium' if player_stats.loc[player, 'Total Minutes'] > 200 else 'low'
        }
    return coef_dict

def plot_player_effects(trace, player_names, team, year):
    """Visualization of player impacts"""
    beta_samples = trace.posterior['beta'].values.reshape(-1, len(player_names))
    
    means = np.mean(beta_samples, axis=0)
    cis = np.percentile(beta_samples, [2.5, 97.5], axis=0)
    
    # Sort by mean impact
    sort_idx = np.argsort(means)[::-1]
    sorted_names = [player_names[i] for i in sort_idx]
    
    plt.figure(figsize=(12, 8))
    plt.hlines(y=range(len(sort_idx)), 
               xmin=cis[0, sort_idx], 
               xmax=cis[1, sort_idx],
               color='lightblue', alpha=0.7)
    plt.scatter(means[sort_idx], range(len(sort_idx)), 
                color='navy', alpha=0.7)
    plt.axvline(0, color='gray', linestyle='--')
    
    plt.yticks(range(len(sort_idx)), sorted_names)
    plt.xlabel('RAPM (Points per 100 Possessions)')
    plt.title(f'{team} {year} Player Impact Estimates\n(95% Credible Intervals)')
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{team}_{year}_RAPM.png', dpi=300, bbox_inches='tight')
    plt.close()

def results_to_dataframe(rapm_output):
    """Convert RAPM results to analysis-ready DataFrame"""
    all_players = []
    for team_year in rapm_output:
        for player, stats in team_year['players'].items():
            row = {
                'Team': team_year['Team'],
                'Year': team_year['Year'],
                'Player': player,
                **stats
            }
            all_players.append(row)
    
    df = pd.DataFrame(all_players)
    return df.sort_values(['Team', 'Year', 'mean'], ascending=[True, True, False])

# Example usage
enhanced_results = enhanced_bayesian_orapm(list(range(1997, 2025)))
results_df = results_to_dataframe(enhanced_results)

Initializing NUTS using adapt_diag...


Running informed model for NYL 1997


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_league, z, sigma_likelihood]


Output()

Sampling 4 chains for 3_000 tune and 3_000 draw iterations (12_000 + 12_000 draws total) took 3 seconds.
Initializing NUTS using adapt_diag...


...

Running informed model for LAS 2024


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_league, z, sigma_likelihood]


Output()

Sampling 4 chains for 3_000 tune and 3_000 draw iterations (12_000 + 12_000 draws total) took 3 seconds.



Skipped/Problematic Cases:
- MIA 2002 (cannot convert the series to <class 'int'>)
- DET 2004 (Chain 3 failed with: Bad initial energy: SamplerWarning(kind=<WarningType.BAD_ENERGY: 8>, message='Bad initial energy, check any log probabilities that are inf or -inf, nan or very small:\n[nan]\n.Try model.debug() to identify parametrization problems.', level='critical', step=0, exec_info=None, extra=None, divergence_point_source=None, divergence_point_dest=None, divergence_info=None))


In [45]:
results_df.head(20)

,Team,Year,Player,mean,lower,upper,minutes,reliability
1901,ATL,2008,SANCHO LYTTLE,0.278579,-5.672444,6.431023,488,medium
1902,ATL,2008,ANN STROTHER,0.179592,-5.483297,6.687201,159,low
1897,ATL,2008,KATIE MATTERA,-1.142528,-5.833157,4.471376,462,medium
1894,ATL,2008,IVORY LATTA,-2.118324,-6.214587,2.411846,958,high
1896,ATL,2008,JENNIFER LACY,-2.779149,-7.388677,3.463036,603,high
1893,ATL,2008,BETTY LENNOX,-3.607114,-7.626661,0.956681,1009,high
1895,ATL,2008,MICHELLE SNOW,-3.770438,-9.400386,1.871160,856,high
1904,ATL,2008,ERIKA DE SOUZA,-4.058867,-10.989156,2.923842,277,medium
1900,ATL,2008,IZIANE CASTRO MARQUES,-4.798286,-10.730817,0.926336,669,high
1899,ATL,2008,TAMERA YOUNG,-6.871500,-11.673798,-2.545068,745,high


In [46]:
best_seasons = results_df.sort_values(by = 'mean', ascending = False)
best_seasons[best_seasons['reliability'] == 'high'].head(20)

,Team,Year,Player,mean,lower,upper,minutes,reliability
3461,WAS,2019,ELENA DELLE DONNE,17.711563,5.772934,29.746364,902,high
812,CHA,2002,KELLY MILLER,12.712048,5.455889,20.460723,553,high
4108,LVA,2023,KELSEY PLUM,12.465513,5.747872,19.162767,1263,high
4105,LVA,2023,JACKIE YOUNG,11.615646,5.137697,18.390475,1260,high
1481,SEA,2006,LAUREN JACKSON,11.600106,7.009929,16.937807,852,high
2382,MIN,2012,TAJ MCWILLIAMS-FRANKLIN,11.411396,4.333919,18.407800,877,high
2529,MIN,2013,SEIMONE AUGUSTUS,10.998216,3.718182,18.335784,923,high
2910,LAS,2015,CANDACE PARKER,10.514702,5.338541,15.901133,550,high
1434,PHO,2006,DIANA TAURASI,10.418832,3.346430,17.974781,1112,high
1568,SEA,2007,IZIANE CASTRO MARQUES,10.378779,2.661859,18.064228,962,high


In [47]:
results_df[results_df['Player'] == "BREANNA STEWART"]

,Team,Year,Player,mean,lower,upper,minutes,reliability
4055,NYL,2023,BREANNA STEWART,7.057696,1.494705,12.362627,1364,high
4264,NYL,2024,BREANNA STEWART,3.215000,-2.658596,8.353799,1242,high
2959,SEA,2016,BREANNA STEWART,3.049319,0.190980,5.858268,1179,high
3120,SEA,2017,BREANNA STEWART,4.413852,-1.038009,9.868260,1085,high
3274,SEA,2018,BREANNA STEWART,4.875107,-2.867136,10.734951,1074,high
3605,SEA,2020,BREANNA STEWART,3.002349,-3.893162,8.422576,608,high
3765,SEA,2021,BREANNA STEWART,4.526702,-1.455189,9.615928,932,high
3918,SEA,2022,BREANNA STEWART,4.215685,1.081814,7.333280,1050,high


In [48]:
results_df.loc[(results_df['Year'] == 2024)  & (results_df['Team'] == 'IND')]

,Team,Year,Player,mean,lower,upper,minutes,reliability
4306,IND,2024,LEXIE HULL,7.498607,2.278973,12.981453,669,high
4308,IND,2024,TEMI FÁGBÉNLÉ,2.052571,-4.988233,8.932748,415,medium
4300,IND,2024,ALIYAH BOSTON,1.244221,-4.018611,6.015857,1236,high
4301,IND,2024,CAITLIN CLARK,1.200426,-3.691548,5.969418,1416,high
4302,IND,2024,KELSEY MITCHELL,1.044940,-4.350380,5.622324,1280,high
4309,IND,2024,DAMIRIS DANTAS,-0.695024,-7.670149,5.449941,210,medium
4303,IND,2024,NALYSSA SMITH,-1.739876,-6.946205,3.252393,988,high
4310,IND,2024,GRACE BERGER,-2.550520,-9.675400,4.311110,102,low
4307,IND,2024,KRISTY WALLACE,-4.713100,-10.696553,0.640342,447,medium
4305,IND,2024,KATIE LOU SAMUELSON,-5.697900,-11.568595,-0.660490,673,high


In [49]:
high_results0 = results_df[results_df['reliability'] == 'high']
high_results0.groupby('Player')['mean'].mean().sort_values(ascending = False)[:20]

Player
MONICA LAMB          8.646073
ELENA DELLE DONNE    7.433264
SATOU SABALLY        7.354552
TASHA HUMPHREY       7.096021
MAYA MOORE           5.841501
JONQUEL JONES        5.646965
ANDREA GARNER        5.295402
HAIXIA ZHENG         5.289805
CYNTHIA COOPER       5.150782
CHENNEDY CARTER      5.115991
KIM PERROT           4.634666
LAUREN JACKSON       4.626392
LATOYA SANDERS       4.512347
BRIONNA JONES        4.414011
NIELE IVEY           4.407637
COURTNEY PARIS       4.299937
BREANNA STEWART      4.294464
KELSEY PLUM          4.261421
JACKIE YOUNG         4.153491
LEONIE FIEBICH       3.903866
Name: mean, dtype: float64

In [50]:
results_df.to_csv('Bayesian_ORAPM.csv', index = False)

In [77]:
def enhanced_bayesian_drapm(years):
    RAPM = []
    problematic_cases = []
    
    for year in years:
        try:
            lineup_DF = pd.read_csv(f"/Users/austincoffelt/Documents/WNBA_lineups/wnba_lineups_{year}_complete.csv")
            players_DF = pd.read_csv(f"/Users/austincoffelt/Documents/WNBA_players/wnba_players_{year}.csv", dtype={'Total Minutes': float, 'relOffRtg': float})
            players_DF = players_DF.drop_duplicates()

            relative_ratings(lineup_DF, players_DF, year)

            teams = list(lineup_DF['Team'].unique())
        
            for team in teams:
                try:
                    lineup_df = lineup_DF[lineup_DF['Team'] == team]
                    lineup_df = lineup_df[lineup_df['Minutes'] >= 2]
                    players_df = players_DF[players_DF['Team'] == team].copy()
                    
                    # Ensure we have relOffRtg for all players
                    assert 'relDefRtg' in players_df.columns, "Missing relDefRtg in players_df"
                    
                    result_df = create_fullname_dummies(lineup_df.copy(), players_df, team, year)
                    player_columns = list(players_df['Name'])
                    result_df = result_df[player_columns + ['relDefRtg']].dropna(subset=['relDefRtg'])
                    
                    if len(result_df) < 20:
                        problematic_cases.append(f"{team} {year} (only {len(result_df)} lineups)")
                        continue
                    
                    X = result_df.drop('relDefRtg', axis=1).values
                    y = result_df['relDefRtg'].values
                    player_names = result_df.drop('relDefRtg', axis=1).columns.tolist()
                    
                    try:
                        print(f"Running informed model for {team} {year}")
                        trace = informed_model(X, y, player_names, players_df)
                    except Exception as e:
                        print(f"Informed model failed, trying robust version: {str(e)}")
                        trace = robust_informed_model(X, y, player_names, players_df)
                    
                    coef_dict = process_results(trace, team, year, player_names, players_df)
                    RAPM.append(coef_dict)
                    plot_player_effects(trace, player_names, team, year)
                    
                except Exception as e:
                    problematic_cases.append(f"{team} {year} ({str(e)})")
                    continue
    
        except Exception as e:
            problematic_cases.append(f"Year {year} failed: {str(e)}")
            continue
            
    if problematic_cases:
        print("\nSkipped/Problematic Cases:")
        for case in problematic_cases:
            print(f"- {case}")
    
    return RAPM

def informed_model(X, y, player_names, players_df):
    """Model using pre-calculated relOffRtg as priors"""
    with pm.Model() as model:
        # Get player priors and minutes
        player_priors = players_df.set_index('Name')['relDefRtg'].reindex(player_names).values
        player_minutes = players_df.set_index('Name')['Total Minutes'].reindex(player_names).fillna(1).values
        
        # Hierarchical variance
        sigma_league = pm.HalfNormal('sigma_league', sigma=5)
        
        # Player effects centered around their relOffRtg
        z = pm.Normal('z', mu=0, sigma=1, shape=len(player_names))
        prior_strength = np.log1p(player_minutes) / 2
        
        beta = pm.Deterministic('beta', player_priors + z * sigma_league / prior_strength)
        mu = pm.math.dot(X, beta)
        
        # Likelihood
        sigma_likelihood = pm.HalfNormal('sigma_likelihood', sigma=5)
        pm.Normal('obs', mu=mu, sigma=sigma_likelihood, observed=y)
        
        trace = pm.sample(
            3000, tune=3000,
            chains=4,
            target_accept=0.95,
            init='adapt_diag'
        )
    return trace

def robust_informed_model(X, y, player_names, players_df):
    """Robust version with Student-T likelihood and parameter constraints"""
    with pm.Model() as model:
        player_priors = players_df.set_index('Name')['relDefRtg'].reindex(player_names).values
        player_minutes = players_df.set_index('Name')['Total Minutes'].reindex(player_names).fillna(1).values
        
        sigma_league = pm.HalfNormal('sigma_league', sigma=5)
        
        z = pm.Normal('z', mu=0, sigma=1, shape=len(player_names))
        prior_strength = np.log1p(player_minutes) / 2
        
        # Constrained player effects
        beta = pm.Deterministic('beta',
                              pm.math.clip(
                                  player_priors + z * sigma_league / prior_strength,
                                  -25, 25  # Reasonable bounds for basketball RAPM
                              ))
        mu = pm.math.dot(X, beta)
        
        # Robust likelihood
        sigma_likelihood = pm.HalfNormal('sigma_likelihood', sigma=5)
        pm.StudentT('obs', nu=4, mu=mu, sigma=sigma_likelihood, observed=y)
        
        trace = pm.sample(
            2000, tune=2000,
            chains=4,
            target_accept=0.95,
            init='adapt_diag'
        )
    return trace

def process_results(trace, team, year, player_names, players_df):
    """Your existing results processing logic"""
    coef_dict = {
        'Team': team,
        'Year': year,
        'players': {},
        'diagnostics': az.summary(trace, var_names=['sigma_league', 'sigma_likelihood']).to_dict()
    }
    
    beta_samples = trace.posterior['beta'].values.reshape(-1, len(player_names))
    player_stats = players_df.set_index('Name')[['Total Minutes']].reindex(player_names)
    
    for i, player in enumerate(player_names):
        samples = beta_samples[:, i]
        coef_dict['players'][player] = {
            'mean': float(np.mean(samples)),
            'lower': float(np.percentile(samples, 2.5)),
            'upper': float(np.percentile(samples, 97.5)),
            'minutes': int(player_stats.loc[player, 'Total Minutes']),
            'reliability': 'high' if player_stats.loc[player, 'Total Minutes'] > 500 else 
                          'medium' if player_stats.loc[player, 'Total Minutes'] > 200 else 'low'
        }
    return coef_dict

def plot_player_effects(trace, player_names, team, year):
    """Visualization of player impacts"""
    beta_samples = trace.posterior['beta'].values.reshape(-1, len(player_names))
    
    means = np.mean(beta_samples, axis=0)
    cis = np.percentile(beta_samples, [2.5, 97.5], axis=0)
    
    # Sort by mean impact
    sort_idx = np.argsort(means)[::-1]
    sorted_names = [player_names[i] for i in sort_idx]
    
    plt.figure(figsize=(12, 8))
    plt.hlines(y=range(len(sort_idx)), 
               xmin=cis[0, sort_idx], 
               xmax=cis[1, sort_idx],
               color='lightblue', alpha=0.7)
    plt.scatter(means[sort_idx], range(len(sort_idx)), 
                color='navy', alpha=0.7)
    plt.axvline(0, color='gray', linestyle='--')
    
    plt.yticks(range(len(sort_idx)), sorted_names)
    plt.xlabel('RAPM (Points per 100 Possessions)')
    plt.title(f'{team} {year} Player Impact Estimates\n(95% Credible Intervals)')
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{team}_{year}_RAPM.png', dpi=300, bbox_inches='tight')
    plt.close()

def results_to_dataframe(rapm_output):
    """Convert RAPM results to analysis-ready DataFrame"""
    all_players = []
    for team_year in rapm_output:
        for player, stats in team_year['players'].items():
            row = {
                'Team': team_year['Team'],
                'Year': team_year['Year'],
                'Player': player,
                **stats
            }
            all_players.append(row)
    
    df = pd.DataFrame(all_players)
    return df.sort_values(['Team', 'Year', 'mean'], ascending=[True, True, False])

# Example usage
enhanced_results_1 = enhanced_bayesian_drapm(list(range(1997, 2025)))
results_df_1 = results_to_dataframe(enhanced_results_1)

Initializing NUTS using adapt_diag...


Running informed model for NYL 1997


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_league, z, sigma_likelihood]


Output()

Sampling 4 chains for 3_000 tune and 3_000 draw iterations (12_000 + 12_000 draws total) took 3 seconds.
Initializing NUTS using adapt_diag...


...

Running informed model for LAS 2024


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_league, z, sigma_likelihood]


Output()

Sampling 4 chains for 3_000 tune and 3_000 draw iterations (12_000 + 12_000 draws total) took 3 seconds.



Skipped/Problematic Cases:
- MIA 2002 (cannot convert the series to <class 'int'>)
- DET 2004 (Chain 1 failed with: Bad initial energy: SamplerWarning(kind=<WarningType.BAD_ENERGY: 8>, message='Bad initial energy, check any log probabilities that are inf or -inf, nan or very small:\n[nan]\n.Try model.debug() to identify parametrization problems.', level='critical', step=0, exec_info=None, extra=None, divergence_point_source=None, divergence_point_dest=None, divergence_info=None))


In [78]:
results_df_1.head(20)

,Team,Year,Player,mean,lower,upper,minutes,reliability
2016,ATL,2008,SANCHO LYTTLE,5.152357,-1.015136,11.249099,488,medium
2012,ATL,2008,KATIE MATTERA,4.210938,-0.549357,10.165666,462,medium
2018,ATL,2008,CHIOMA NNAMAKA,3.272632,-4.163810,10.408248,84,low
2010,ATL,2008,MICHELLE SNOW,2.042688,-3.670377,7.759967,856,high
2017,ATL,2008,ANN STROTHER,1.563305,-4.377282,7.855030,159,low
2013,ATL,2008,KRISTIN HAYNIE,0.853732,-4.039785,5.757644,485,medium
2014,ATL,2008,TAMERA YOUNG,-2.395514,-6.717654,3.080858,745,high
2011,ATL,2008,JENNIFER LACY,-4.976409,-9.653265,0.202416,603,high
2008,ATL,2008,BETTY LENNOX,-5.426278,-9.463004,-0.510684,1009,high
2015,ATL,2008,IZIANE CASTRO MARQUES,-6.463729,-12.317717,-0.487635,669,high


In [79]:
best_seasons1 = results_df_1.sort_values(by = 'mean', ascending = False)
best_seasons1[best_seasons1['reliability'] == 'high'].head(20)

,Team,Year,Player,mean,lower,upper,minutes,reliability
124,HOU,1998,MONICA LAMB,14.166692,5.588360,23.450319,648,high
515,MIA,2000,MARLIES ASKAMP,13.215867,6.855067,20.485395,870,high
2553,PHO,2012,NAKIA SANFORD,11.499713,4.240822,19.224884,501,high
3101,MIN,2016,RENEE MONTGOMERY,11.054438,3.907009,18.610589,656,high
2257,WAS,2010,NAKIA SANFORD,10.691611,3.608920,17.392849,608,high
3756,IND,2020,TIFFANY MITCHELL,10.112483,2.230512,18.596470,501,high
1492,IND,2006,TAMIKA CATCHINGS,9.911726,3.520774,16.709796,1068,high
2217,ATL,2010,ALISON BALES,9.715932,3.152607,16.236371,530,high
530,CLE,2000,ANN WAUTERS,9.450525,3.857250,13.458603,573,high
3867,LVA,2021,LIZ CAMBAGE,9.387659,3.119931,16.070276,595,high


In [80]:
results_df_1[results_df_1['Player'] == "LAUREN JACKSON"]

,Team,Year,Player,mean,lower,upper,minutes,reliability
735,SEA,2001,LAUREN JACKSON,2.058178,-0.743131,5.674400,1000,high
930,SEA,2002,LAUREN JACKSON,4.479369,-0.080782,8.337321,879,high
1097,SEA,2003,LAUREN JACKSON,0.153396,-3.179236,2.589922,1108,high
1172,SEA,2004,LAUREN JACKSON,4.791884,0.137988,9.379611,1069,high
1357,SEA,2005,LAUREN JACKSON,0.035272,-2.027063,2.340135,1176,high
1596,SEA,2006,LAUREN JACKSON,1.503784,-1.483787,4.320964,852,high
1688,SEA,2007,LAUREN JACKSON,2.446225,-0.027528,5.911537,1019,high
1908,SEA,2008,LAUREN JACKSON,5.685105,0.852501,10.236173,693,high
2027,SEA,2009,LAUREN JACKSON,2.892411,-2.804289,8.695641,842,high
2201,SEA,2010,LAUREN JACKSON,2.336739,-4.147651,8.117717,992,high


In [81]:
high_results = results_df_1[results_df_1['reliability'] == 'high']
high_results.groupby('Player')['mean'].mean().sort_values(ascending = False)[:20]

Player
MONICA LAMB         14.166692
ANGEL GOODRICH       7.269869
JADE MELBOURNE       6.776425
SLOBODANKA TUVIC     6.762139
CELINE DUMERC        6.452586
MERY ANDRADE         6.179034
RUSHIA BROWN         5.980796
SUE WICKS            5.941442
LEONIE FIEBICH       5.876996
ERIN PHILLIPS        5.776640
BRIONNA JONES        5.731967
LI MENG              5.710965
TE'A COOPER          5.682964
CHARMIN SMITH        5.613281
VANESSA HAYDEN       5.548229
SHARON MANNING       5.516476
NAKIA SANFORD        5.450236
JONQUEL JONES        5.372906
ANGIE POTTHOFF       5.336648
LAURA HARPER         5.182402
Name: mean, dtype: float64

In [82]:
results_df_1[results_df_1['Player'] == "JONQUEL JONES"]

,Team,Year,Player,mean,lower,upper,minutes,reliability
3153,CON,2016,JONQUEL JONES,6.381837,3.361901,9.749263,479,medium
3245,CON,2017,JONQUEL JONES,1.858267,-2.854696,5.252112,969,high
3465,CON,2018,JONQUEL JONES,5.692058,0.869446,9.463791,697,high
3550,CON,2019,JONQUEL JONES,5.965752,3.540168,8.927966,979,high
3855,CON,2021,JONQUEL JONES,8.461304,1.266308,15.836484,855,high
4021,CON,2022,JONQUEL JONES,7.643491,1.398191,13.923975,871,high
4171,NYL,2023,JONQUEL JONES,2.588947,-3.364641,7.482260,1000,high
4378,NYL,2024,JONQUEL JONES,5.400521,-0.698502,11.669706,1166,high


In [83]:
results_df_1.to_csv('Bayesian_DRAPM.csv', index = False)